In [1]:
import os
from dotenv import load_dotenv
from functools import lru_cache
from crewai import LLM
from datetime import datetime
# Load environment variables from .env file
load_dotenv()

# Set the environment variables
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GEMINI1_API_KEY = os.getenv("GEMINI1_API_KEY")

OPENROUTER_API_KEY2=os.getenv("OPENROUTER_API_KEY2")
OPENAI_API_BASE=os.getenv("OPENAI_API_BASE")

os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

print("API Keys loaded successfully.")

@lru_cache(maxsize=1)
def initialize_llm():
    return LLM(
        model="openrouter/z-ai/glm-4.5-air:free",
        api_key=OPENROUTER_API_KEY2,
        base_url=os.getenv("OPENAI_API_BASE", "https://openrouter.ai/api/v1"),
        temperature=0.4,        # lower randomness for agentic use            # enable streaming if helpful
    )

@lru_cache(maxsize=1)
def initialize_llm1():
    """Initialize and cache the LLM instance to avoid repeated initializations."""
    return LLM(
        model="gemini/gemini-2.0-flash",
        provider="google",
        api_key=GEMINI_API_KEY
    )

import requests
import json
from crewai.tools import tool          # decorator
from crewai_tools import SerperDevTool # web-search tool

# Initialize the web search tool
search_tool = SerperDevTool()

# Tool 1: Human Input Tool
# This tool pauses the execution and asks for human input.
@tool("Human Input Tool")
def human_input_tool(question: str) -> str:
    """Asks a human for input. The agent should use this to ask for a budget if one is not provided."""
    return input(f"\n{question}\n")

def geocode_city(city: str) -> tuple[float, float] | None:
    url = "https://geocoding-api.open-meteo.com/v1/search"
    resp = requests.get(url, params={"name": city, "count": 1, "language": "en"})
    resp.raise_for_status()
    results = resp.json().get("results")
    if results:
        return results[0]["latitude"], results[0]["longitude"]
    return None

# Tool 2: Weather Tool (Updated for Forecast)
bad_weather_codes = [51, 53, 55, 56, 57, 61, 63, 65, 66, 67, 71, 73, 75, 77, 80, 81, 82, 85, 86, 95, 96, 99]
desc_map = {
    0: "clear sky", 1: "mainly clear", 2: "partly cloudy", 3: "overcast", 45: "foggy", 51: "light drizzle", 
    61: "rain", 71: "snow", 95: "thunderstorm"
}

@tool("Weather Tool")
def open_meteo_weather_tool(city: str, start_date: str, end_date: str) -> str:
    """Returns weather forecast for a city between start_date and end_date using Open-Meteo."""
    coords = geocode_city(city)
    if not coords:
        return f"Sorry, I couldn’t find coordinates for {city}."
    lat, lon = coords
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "daily": "temperature_2m_max,temperature_2m_min,weathercode",
        "start_date": start_date,
        "end_date": end_date,
        "timezone": "auto"
    }
    try:
        r = requests.get(url, params=params, timeout=8)
        r.raise_for_status()
        data = r.json()
        daily = data["daily"]
        forecast_lines = [f"Weather forecast for {city.title()} from {start_date} to {end_date}:"]
        bad_weather_dates = []
        for i in range(len(daily["time"])):
            date = daily["time"][i]
            max_temp = daily["temperature_2m_max"][i]
            min_temp = daily["temperature_2m_min"][i]
            code = daily["weathercode"][i]
            desc = desc_map.get(code, "unknown")
            forecast_lines.append(f"- {date}: {min_temp}°C to {max_temp}°C, {desc}")
            if code in bad_weather_codes:
                bad_weather_dates.append(date)
        if bad_weather_dates:
            forecast_lines.append("\nNote: Bad weather (rain, snow, or thunderstorms) expected on: " + ", ".join(bad_weather_dates))
        return "\n".join(forecast_lines)
    except Exception as e:
        return f"Error fetching Open-Meteo data: {e}"

# Tool 3: Currency Conversion Tool
@tool("Currency Conversion Tool")
def currency_conversion_tool(from_currency: str, to_currency: str) -> str:
    """Converts a value from one currency to another using a free API."""
    try:
        # Using exchangerate-api.com's free tier
        url = f"https://open.er-api.com/v6/latest/{from_currency}"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        rate = data['rates'][to_currency]
        return f"The conversion rate from {from_currency} to {to_currency} is {rate:.2f}."
    except Exception as e:
        return f"Error converting currency: {e}. Ensure currency codes are correct (e.g., USD, EUR, LKR)."


print("Tools created successfully.")

# Define country to currency mapping
country_to_currency = {
    'Sri Lanka': 'LKR',
    'United States': 'USD',
    'United Kingdom': 'GBP',
    # Add more as needed
}

# --- Define User Inputs ---
location = 'Mirissa, Sri Lanka'
interests = 'entertainment, beach and affordable villa with pool. we need lunch dinner breakfast to eat in an affordable way and also we will be taking drinks, 1 alcohol bottle for 4 of us'
budget = '250 USD'
num_people = 4 
travel_dates = '2025-08-05 to 2025-08-06'  # Static travel dates
preferred_currency = ''  # Leave empty for local currency, or set to 'GBP', etc.

# Determine local currency
country = location.split(',')[-1].strip()
local_currency = country_to_currency.get(country, 'USD')
target_currency = preferred_currency if preferred_currency else local_currency

def calculate_nights(dates: str) -> int:
    """Calculates the number of nights for a given date range."""
    try:
        start_str, end_str = dates.split(' to ')
        start_date = datetime.strptime(start_str.strip(), '%Y-%m-%d')
        end_date = datetime.strptime(end_str.strip(), '%Y-%m-%d')
        # The number of nights is the difference in days
        num_nights = (end_date - start_date).days
        return max(0, num_nights)  # Return 0 if dates are invalid or same day
    except (ValueError, IndexError):
        return 0

# Determine if accommodation is needed
# Calculate the number of nights
num_nights = calculate_nights(travel_dates)

# Determine if accommodation is needed and create the instruction
accommodation_instruction = ""
if num_nights > 0:
    accommodation_instruction = f"""
    **Crucially, you MUST research and suggest one suitable accommodation for a {num_nights}-night stay.**
    Given the interest in 'villa', prioritize finding a villa for {num_people} people.
    The total cost of the accommodation for the entire {num_nights}-night stay must be factored into the total budget of {budget}.
    """

# Initialize LLM
llm_model = initialize_llm()
llm1_model = initialize_llm1()

from crewai import Agent

# Agent 2: Local Data Agent
local_data_agent = Agent(
    role="Local Data Specialist",
    goal="Fetch weather and currency data for the travel destination.",
    backstory="An analyst providing real-time travel insights.",
    tools=[open_meteo_weather_tool, currency_conversion_tool],
    llm=llm1_model,
    verbose=True
)

# Agent 3: Web Search Agent (City Expert)
city_expert_agent = Agent(
    role='Expert City Researcher',
    goal='Efficiently find a specific number of activities and accommodation within a budget.',
    backstory='A travel enthusiast who finds the best spots tailored to your needs, focusing on speed and accuracy.',
    tools=[search_tool],
    llm=llm_model,
    verbose=True,
    max_iter=10,  # Hard limit on the number of execution loops (thinking -> tool -> observation)
    allow_delegation=False
)

# Agent 4: Budget Verifier Agent
budget_verifier_agent = Agent(
    role='Budget Verification Analyst',
    goal='Critically analyze the researched activities and their estimated costs against the user-provided budget. Provide a clear "go" or "no-go" verdict with justification.',
    backstory='A meticulous financial analyst with a knack for sniffing out hidden costs and ensuring travel plans are financially sound. You are firm but fair.',
    tools=[],
    llm=llm1_model,
    allow_delegation=False,
    verbose=True
)

# Agent 5: Travel Concierge Agent
travel_concierge_agent = Agent(
    role='Head Travel Concierge',
    goal='Synthesize all gathered information into a cohesive, beautifully formatted travel itinerary with weather insights and converted costs.',
    backstory='A world-class concierge from a five-star hotel, known for creating personalized and delightful travel experiences.',
    tools=[currency_conversion_tool],  # Added for cost conversion
    llm=llm1_model,
    allow_delegation=False,
    verbose=True
)

print("Agents defined successfully.")

from crewai import Task

# Task 2: Get local data (weather forecast and currency conversion)
task_get_local_data = Task(
    description=f"""Fetch the weather forecast and the USD to local currency conversion rate for the trip to {location}.

    The travel dates are: {travel_dates}.

    You MUST use the Weather Tool with the exact start and end dates from the provided travel dates.
    Do not use 'today' or any other made-up date. First, call the currency conversion tool, then call the weather tool.
    """,
    expected_output="A summary of the weather forecast for the specified dates and the USD to local currency conversion rate.",
    agent=local_data_agent
)

# Task 3: Find city information
task_find_city_info = Task(
    description=f"""
    For a group of {num_people} traveling to {location}, create a focused travel plan based on a total budget of {budget} and interests in '{interests}'.

    **Your instructions are to be highly efficient. Aim to use the web search tool no more than 2-3 times.**

    Your research output MUST contain the following specific items:
    1.  **Exactly 2-3** top attractions or restaurants or entertainment places that match the interests.
    2.  {accommodation_instruction} 

    Once you have gathered information for these items, consolidate them into a single list with descriptions and estimated USD costs for the group.
    The TOTAL estimated cost of all researched items must not exceed the budget.
    Do not search for more items than requested. Your final answer should be the complete, consolidated list.
    """,
    expected_output="A concise list of 2-3 attractions/restaurants/entertainment places and accommodation (if required), with descriptions and a total estimated cost in USD. The agent should stop once these items are found.",
    agent=city_expert_agent,
    context=[task_get_local_data]
)

# Task 4: Verify the budget
task_verify_budget = Task(
    description=f"Analyze the research from the city expert for a group of {num_people}. The total budget is {budget}. Sum up the total estimated cost of ALL items (activities and accommodation) provided by the researcher. Compare this total to the budget. Provide a clear 'go' or 'no-go' verdict with a brief justification.",
    expected_output="A budget feasibility verdict (Go/No-Go) with the total estimated cost compared against the total budget.",
    agent=budget_verifier_agent,
    context=[task_find_city_info]
)

# Task 5: Compile the final report
task_compile_report = Task(
    description=f"""
    Create a final, human-readable travel itinerary for {num_people} people for a trip to {location} from {travel_dates}.
    
    Your report must:
    1. Synthesize the list of activities and accommodation (if provided in the context) into a cohesive plan.
    2. Include the budget verification verdict.
    3. Convert all USD costs to {target_currency} using the Currency Conversion Tool. Display only the final costs in {target_currency}.
    4. Incorporate weather insights. If bad weather is predicted, suggest alternative indoor activities or mention the flexibility to adjust dates.
    5. Format the entire output as a beautiful and exciting markdown report.
    """,
    expected_output=f"A complete, beautifully formatted markdown report with a travel plan, budget analysis, and weather insights. All costs must be in {target_currency}.",
    agent=travel_concierge_agent,
    context=[task_verify_budget, task_get_local_data, task_find_city_info]
)

print("Tasks created successfully.")

from crewai import Crew, Process
from IPython.display import Markdown

# Create the Crew
travel_crew = Crew(
    agents=[local_data_agent, city_expert_agent, budget_verifier_agent, travel_concierge_agent],
    tasks=[task_get_local_data, task_find_city_info, task_verify_budget, task_compile_report],
    process=Process.sequential,
    verbose=True
)

# Kick off the crew's work!
result = travel_crew.kickoff()

# Print the final result
if hasattr(result, 'raw') and isinstance(result.raw, str):
    display(Markdown(result.raw))
else:
    print(f"Error: Expected a CrewOutput object with a 'raw' attribute containing a string, but got {type(result)}")



API Keys loaded successfully.


/Users/nithilathawalampitiya/Documents/Projects/TravelAgent/myenv-crew/lib/python3.12/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


Tools created successfully.
Agents defined successfully.
Tasks created successfully.


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 39f6d7db-1174-44b3-88e0-165aa457626b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Local Data Specialist                                                                                   │
│                                                                                                                 │
│  Task: Fetch the weather forecast and the USD to local currency conversion rate for the trip to Mirissa, Sri    │
│  Lanka.                                                                                                         │
│                                                                                                                 │
│      The travel dates are: 2025-08-05 to 2025-08-06.                                                            │
│                                                                                                                 │
│      You MUST use the Weather Tool with the exact start and end dates from the provided travel dates.           │
│      Do not use 'today' or any other made-up date. First, call the currency conversion tool, then call the      │
│  weather tool.                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  The conversion rate from USD to LKR is 301.90.                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Head Travel Concierge                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # Mirissa Getaway: A Sri Lankan Adventure (August 5-6, 2025)                                                   │
│                                                                                                                 │
│  ## Trip Overview                                                                                               │
│                                                                                                                 │
│  Get ready for an unforgettable 2-day escape to Mirissa, Sri Lanka! This itinerary is designed for a group of   │
│  4, balancing relaxation, cultural experiences, and delicious food, all while staying within budget.            │
│                                                                                                                 │
│  **Budget Verdict:** Go! (Total estimated cost: $205 USD, within the $250 USD budget)                           │
│                                                                                                                 │
│  **All costs are shown in LKR (Sri Lankan Rupees).**                                                            │
│                                                                                                                 │
│  ## Day 1: August 5, 2025 - Beach Bliss and Sunset Views                                                        │
│                                                                                                                 │
│  **Weather:** 26.4°C to 29.4°C, overcast                                                                        │
│                                                                                                                 │
│  | Time          | Activity                                  | Description                                      │
│  | Estimated Cost (LKR) |                                                                                       │
│  |---------------|-------------------------------------------|------------------------------------------------  │
│  --------------------------------------|----------------------|                                                 │
│  | Morning       | Arrival & Check-in at Kumbura Family Villa with Pool | Settle into your charming villa with  │
│  a pool and enjoy the beautiful surroundings. | 36,228.00            |                                          │
│  | Afternoon     | Mirissa Beach                             | Relax on the golden sands, swim in the           │
│  turquoise waters, and soak up the sun.        | 0.00                 |                                         │
│  | Lunch         | Petti Petti Mirissa                       | Indulge in authentic Sri Lankan cuisine at this  │
│  affordable local restaurant.           | 7,547.50             |                                                │
│  | Late Afternoon | Coconut Hill Sunset Viewpoint            | Witness a breathtaking sunset over the Indian    │
│  Ocean from this iconic viewpoint.       | 0.00                 |                                               │
│  | Dinner        | Local Seafood Restaurant                  | Feast on fresh, delicious seafood at a           │
│  restaurant near the beach.                      | 10,5

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ef013faf-9503-4797-915e-9da26aaec1f8                                                                     │
│  Agent: Head Travel Concierge                                                                                   │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 39f6d7db-1174-44b3-88e0-165aa457626b                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # Mirissa Getaway: A Sri Lankan Adventure (August 5-6, 2025)                                                   │
│                                                                                                                 │
│  ## Trip Overview                                                                                               │
│                                                                                                                 │
│  Get ready for an unforgettable 2-day escape to Mirissa, Sri Lanka! This itinerary is designed for a group of   │
│  4, balancing relaxation, cultural experiences, and delicious food, all while staying within budget.            │
│                                                                                                                 │
│  **Budget Verdict:** Go! (Total estimated cost: $205 USD, within the $250 USD budget)                           │
│                                                                                                                 │
│  **All costs are shown in LKR (Sri Lankan Rupees).**                                                            │
│                                                                                                                 │
│  ## Day 1: August 5, 2025 - Beach Bliss and Sunset Views                                                        │
│                                                                                                                 │
│  **Weather:** 26.4°C to 29.4°C, overcast                                                                        │
│                                                                                                                 │
│  | Time          | Activity                                  | Description                                      │
│  | Estimated Cost (LKR) |                                                                                       │
│  |---------------|-------------------------------------------|------------------------------------------------  │
│  --------------------------------------|----------------------|                                                 │
│  | Morning       | Arrival & Check-in at Kumbura Family Villa with Pool | Settle into your charming villa with  │
│  a pool and enjoy the beautiful surroundings. | 36,228.00            |                                          │
│  | Afternoon     | Mirissa Beach                             | Relax on the golden sands, swim in the           │
│  turquoise waters, and soak up the sun.        | 0.00                 |                                         │
│  | Lunch         | Petti Petti Mirissa                       | Indulge in authentic Sri Lankan cuisine at this  │
│  affordable local restaurant.           | 7,547.50             |                                                │
│  | Late Afternoon | Coconut Hill Sunset Viewpoint            | Witness a breathtaking sunset over the Indian    │
│  Ocean from this iconic viewpoint.       | 0.00                 |                                               │
│  | Dinner        | Local Seafood Restaurant           

```markdown
# Mirissa Getaway: A Sri Lankan Adventure (August 5-6, 2025)

## Trip Overview

Get ready for an unforgettable 2-day escape to Mirissa, Sri Lanka! This itinerary is designed for a group of 4, balancing relaxation, cultural experiences, and delicious food, all while staying within budget.

**Budget Verdict:** Go! (Total estimated cost: $205 USD, within the $250 USD budget)

**All costs are shown in LKR (Sri Lankan Rupees).**

## Day 1: August 5, 2025 - Beach Bliss and Sunset Views

**Weather:** 26.4°C to 29.4°C, overcast

| Time          | Activity                                  | Description                                                                          | Estimated Cost (LKR) |
|---------------|-------------------------------------------|--------------------------------------------------------------------------------------|----------------------|
| Morning       | Arrival & Check-in at Kumbura Family Villa with Pool | Settle into your charming villa with a pool and enjoy the beautiful surroundings. | 36,228.00            |
| Afternoon     | Mirissa Beach                             | Relax on the golden sands, swim in the turquoise waters, and soak up the sun.        | 0.00                 |
| Lunch         | Petti Petti Mirissa                       | Indulge in authentic Sri Lankan cuisine at this affordable local restaurant.           | 7,547.50             |
| Late Afternoon | Coconut Hill Sunset Viewpoint            | Witness a breathtaking sunset over the Indian Ocean from this iconic viewpoint.       | 0.00                 |
| Dinner        | Local Seafood Restaurant                  | Feast on fresh, delicious seafood at a restaurant near the beach.                      | 10,566.50            |
| Evening       | Drinks at Villa                           | Enjoy local beverages and share a bottle among the group.                             | 4,528.50             |

## Day 2: August 6, 2025 - Drizzle and Departure

**Weather:** 23.8°C to 29.5°C, light drizzle

*Note: Light drizzle is expected today. Consider bringing umbrellas or raincoats.*

| Time          | Activity                                  | Description                                                                                   | Estimated Cost (LKR) |
|---------------|-------------------------------------------|-----------------------------------------------------------------------------------------------|----------------------|
| Morning       | Breakfast at Villa                        | Enjoy a leisurely breakfast at the villa or grab a simple local breakfast from a nearby shop. | 3,019.00             |
| Late Morning    | Relaxing at the Villa or Explore local shops | Enjoy your time at the villa or explore local shops to buy souvenirs. | 0.00   |
| Lunch         |  Enjoy lunch at Villa                     | Talk to villa staff to prepare lunch at the villa. | 0.00 |
| Afternoon     | Check out and Departure                  | Bid farewell to Mirissa and head home with unforgettable memories.                            | 0.00                 |

## Total Estimated Costs (LKR)

*   **Accommodation:** 36,228.00
*   **Dining:** 25,661.50
*   **Drinks:** 4,528.50
*   **Activities:** 0.00

**Grand Total: 66,418.00 LKR**

## Weather Contingency

*   **August 6th:** Light drizzle is predicted. If the rain intensifies, consider spending more time relaxing at the villa, playing games, or reading. Alternatively, explore indoor activities like visiting a local temple or cooking class (may require adjustments to the budget). The dates can also be adjusted slightly if needed to avoid the rain.

## Important Notes

*   This itinerary is a suggestion and can be customized to your preferences.
*   Prices are estimates and may vary.
*   Remember to bring sunscreen, insect repellent, and comfortable shoes.
*   Enjoy your trip to Mirissa!
```

In [2]:
# from functools import lru_cache
# from crewai import LLM
# import os
# from datetime import datetime
# from crewai_tools import SerperDevTool # web-search tool


# OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
# OPENAI_API_BASE=os.getenv("OPENAI_API_BASE")

# os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

# print("API Keys loaded successfully.")

# @lru_cache(maxsize=1)
# def initialize_llm():
#     return LLM(
#         model="openrouter/z-ai/glm-4.5-air:free",
#         api_key=os.getenv("OPENROUTER_API_KEY"),
#         base_url=os.getenv("OPENAI_API_BASE", "https://openrouter.ai/api/v1"),
#         temperature=0.4,        # lower randomness for agentic use            # enable streaming if helpful
#     )

# # Define country to currency mapping
# country_to_currency = {
#     'Sri Lanka': 'LKR',
#     'United States': 'USD',
#     'United Kingdom': 'GBP',
#     # Add more as needed
# }
# # Initialize the web search tool
# search_tool = SerperDevTool()

# location = 'Mirissa, Sri Lanka'
# interests = 'entertainment, beach and affordable villa with pool'
# budget = '250 USD'
# num_people = 4 
# travel_dates = '2025-08-05 to 2025-08-06'  # Static travel dates
# preferred_currency = ''  # Leave empty for local currency, or set to 'GBP', etc.

# # Determine local currency
# country = location.split(',')[-1].strip()
# local_currency = country_to_currency.get(country, 'USD')
# target_currency = preferred_currency if preferred_currency else local_currency

# def calculate_nights(dates: str) -> int:
#     """Calculates the number of nights for a given date range."""
#     try:
#         start_str, end_str = dates.split(' to ')
#         start_date = datetime.strptime(start_str.strip(), '%Y-%m-%d')
#         end_date = datetime.strptime(end_str.strip(), '%Y-%m-%d')
#         # The number of nights is the difference in days
#         num_nights = (end_date - start_date).days
#         return max(0, num_nights)  # Return 0 if dates are invalid or same day
#     except (ValueError, IndexError):
#         return 0

# # Determine if accommodation is needed
# # Calculate the number of nights
# num_nights = calculate_nights(travel_dates)

# # Determine if accommodation is needed and create the instruction
# accommodation_instruction = ""
# if num_nights > 0:
#     accommodation_instruction = f"""
#     **Crucially, you MUST research and suggest one suitable accommodation for a {num_nights}-night stay.**
#     Given the interest in 'villa', prioritize finding a villa for {num_people} people.
#     The total cost of the accommodation for the entire {num_nights}-night stay must be factored into the total budget of {budget}.
#     """

# # Initialize LLM
# llm_model = initialize_llm()

# from crewai import Agent

# # Agent 3: Web Search Agent (City Expert)
# city_expert_agent = Agent(
#     role='Expert City Researcher',
#     goal='Efficiently find a specific number of activities and accommodation within a budget.',
#     backstory='A travel enthusiast who finds the best spots tailored to your needs, focusing on speed and accuracy.',
#     tools=[search_tool],
#     llm=llm_model,
#     verbose=True,
#     max_iter=5,  # Hard limit on the number of execution loops (thinking -> tool -> observation)
#     allow_delegation=False
# )

# from crewai import Task

# # Task 3: Find city information
# task_find_city_info = Task(
#     description=f"""
#     For a group of {num_people} traveling to {location}, create a focused travel plan based on a total budget of {budget} and interests in '{interests}'.

#     **Your instructions are to be highly efficient. Aim to use the web search tool no more than 2-3 times.**

#     Your research output MUST contain the following specific items:
#     1.  **Exactly 2-3** top attractions or restaurants or entertainment places that match the interests.
#     2.  {accommodation_instruction} # This is the dynamic instruction from the previous step

#     Once you have gathered information for these items, consolidate them into a single list with descriptions and estimated USD costs for the group.
#     The TOTAL estimated cost of all researched items must not exceed the budget.
#     Do not search for more items than requested. Your final answer should be the complete, consolidated list.
#     """,
#     expected_output="A concise list of 2-3 attractions/restaurants/entertainment places and accommodation (if required), with descriptions and a total estimated cost in USD. The agent should stop once these items are found.",
#     agent=city_expert_agent
# )

# from crewai import Crew, Process
# from IPython.display import Markdown

# # Create the Crew
# travel_crew = Crew(
#     agents=[city_expert_agent],
#     tasks=[task_find_city_info],
#     process=Process.sequential,
#     verbose=True
# )

# # Kick off the crew's work!
# result = travel_crew.kickoff()